In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
from PIL import Image
import cv2
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import Dataset, DataLoader
from albumentations.core.serialization import Type

In [ ]:
from tqdm import tqdm
import os
import pandas as pd
import zipfile

In [ ]:
CFG = {
    'IMG_SIZE': 512,
    'EPOCHS': 30,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':16,
    'SEED':41
}

In [ ]:
file_path = './content/'
output_unzip = zipfile.ZipFile("/content/drive/MyDrive/open.zip", "r")  # "r": read 모드
output_unzip.extractall(file_path)
output_unzip.close()

In [ ]:
PATH = '/content/drive/MyDrive/train_imgs/'
file_name = os.listdir(PATH)

##증강 사용
##B set Horizontalflip + shiftscalerotate


In [ ]:
hflip_prob= float(0.5)
ssr_prob= float(0.5) 
random_bc_prob= float(0.5)



aug = A.Compose([
                            A.HorizontalFlip(p=hflip_prob),
                            # A.VerticalFlip(),
                            A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=10),
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()

                             ])

aug = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
])





##C set Horizontalflip + shiftscalerotate + eaualization + CLAHE

In [ ]:
hflip_prob= float(0.5)
ssr_prob= float(0.5) 
random_bc_prob= float(0.5)


aug = A.Compose([
                            A.HorizontalFlip(p=hflip_prob),
                            # A.VerticalFlip(),
                            A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=10, p=ssr_prob),
                            # A.Rotate(limit=90, border_mode=cv2.BORDER_CONSTANT,p=0.3),
                            A.CLAHE(clip_limit=(1,4), p = 1),
                            A.equalize,
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()

                             ])


aug = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
])


##D set Horizontalflip + shiftscalerotate + eaualization + CLAHE + Affine + centercrop

In [ ]:
hflip_prob= float(0.5)
ssr_prob= float(0.5) 
random_bc_prob= float(0.5)



aug = A.Compose([
                            A.HorizontalFlip(p=hflip_prob),
                            # A.VerticalFlip(),
                            # A.Rotate(limit=90, border_mode=cv2.BORDER_CONSTANT,p=0.3),
                            # A.RandomResizedCrop(CFG['IMG_SIZE'],CFG['IMG_SIZE'], scale=(0.9,1), p = 1),
                            A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=10, p=ssr_prob),
                            A.CLAHE(clip_limit=(1,4), p = 1),
                            A.equalize,
                            A.CenterCrop(200, 200),
                            A.Affine(always_apply=False, scale = 1.2, p=0.5),
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()

                             ])

aug = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
for i in file_name:
    file_path = '/content/open/train_imgs/'+ i 
    img = cv2.imread(file_path)
    imgg = aug(image = img)
    array = imgg['image']

    for j in range(2): # 증강 사진 개수 늘리는 부분 e.g.)1000장 일 경우 ()안의 수 : 1, 2000장일 경우 ()안의 수 : 2
      i = i.replace('.png', '')
      cv2.imwrite(f'/content/train_img/{i}__C{j}.png', array) # 이미지 저장하는 부분